# Read Data

* Spider Dataset

* https://yale-lily.github.io/spider

In [1]:
from datasets import load_dataset

# Load Spider dataset
ds = load_dataset("spider")
db_ids = ["department_management", "farm", "aircraft", "architecture", "cinema"]

data = []

for db_id in db_ids:
    subset = ds["train"].filter(lambda x: x["db_id"] == db_id)
    questions = [entry["question"] for entry in subset]
    queries = [entry["query"] for entry in subset]

    # Combine into (db_id, question, query) triplets
    entries = [{"db_id": db_id, "question": q, "query": sql} for q, sql in zip(questions, queries)]

    # Add to overall list
    data.extend(entries)

# sqlcoder-7b-2

# Read Data

* Spider Dataset

* https://yale-lily.github.io/spider

In [2]:
import sqlparse
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TRANSFORMERS_CACHE

print(torch.cuda.is_available())
torch.cuda.empty_cache()

available_memory = torch.cuda.get_device_properties(0).total_memory
print(available_memory)

print(TRANSFORMERS_CACHE)

model_name = "defog/sqlcoder-7b-2"
cache_dir = "E:/Data File/transformers.cache"  # Specify your desired cache directory
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)

if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
        cache_dir=cache_dir
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
        cache_dir=cache_dir
    )

True
17170956288
C:\Users\zly20\.cache\huggingface\hub


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
department_prompt = """
### Task
Generate a SQLite query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Pay close attention to the given database schema, note on how they can join together
- You do not need to generate your thought process but just the answer
- Your answer should end with '[/SQL]'

CREATE TABLE IF NOT EXISTS "department" (
"Department_ID" int,
"Name" text,
"Creation" text,
"Ranking" int,
"Budget_in_Billions" real,
"Num_Employees" real,
PRIMARY KEY ("Department_ID")
);

CREATE TABLE IF NOT EXISTS "head" (
"head_ID" int,
"name" text,
"born_state" text,
"age" real,
PRIMARY KEY ("head_ID")
);

CREATE TABLE IF NOT EXISTS "management" (
"department_ID" int,
"head_ID" int,
"temporary_acting" text,
PRIMARY KEY ("Department_ID","head_ID"),
FOREIGN KEY ("Department_ID") REFERENCES `department`("Department_ID"),
FOREIGN KEY ("head_ID") REFERENCES `head`("head_ID")
);

### Answer
Given the database schema, here is the SQLite query that answers [QUESTION]{question}[/QUESTION]
[SQL]
""" 


farm_prompt = """
### Task
Generate a SQLite query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Pay close attention to the given database schema, note on how they can join together
- You do not need to generate your thought process but just the answer
- Your answer should end with '[/SQL]'

CREATE TABLE "city" (
"City_ID" int,
"Official_Name" text,
"Status" text,
"Area_km_2" real,
"Population" real,
"Census_Ranking" text,
PRIMARY KEY ("City_ID")
);

CREATE TABLE "farm" (
"Farm_ID" int,
"Year" int,
"Total_Horses" real,
"Working_Horses" real,
"Total_Cattle" real,
"Oxen" real,
"Bulls" real,
"Cows" real,
"Pigs" real,
"Sheep_and_Goats" real,
PRIMARY KEY ("Farm_ID")
);

CREATE TABLE "farm_competition" (
"Competition_ID" int,
"Year" int,
"Theme" text,
"Host_city_ID" int,
"Hosts" text,
PRIMARY KEY ("Competition_ID"),
FOREIGN KEY (`Host_city_ID`) REFERENCES `city`(`City_ID`)
);


CREATE TABLE "competition_record" (
"Competition_ID" int,
"Farm_ID" int,
"Rank" int,
PRIMARY KEY ("Competition_ID","Farm_ID"),
FOREIGN KEY (`Competition_ID`) REFERENCES `farm_competition`(`Competition_ID`),
FOREIGN KEY (`Farm_ID`) REFERENCES `farm`(`Farm_ID`)
);

### Answer
Given the database schema, here is the SQLite query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""


aircraft_prompt = """
### Task
Generate a SQLite query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Pay close attention to the given database schema, note on how they can join together
- You do not need to generate your thought process but just the answer
- Your answer should end with '[/SQL]'

CREATE TABLE `pilot` (
  `Pilot_Id` int(11) NOT NULL,
  `Name` varchar(50) NOT NULL,
  `Age` int(11) NOT NULL,
  PRIMARY KEY (`Pilot_Id`)
);

CREATE TABLE `aircraft` (
  "Aircraft_ID" int(11) NOT NULL,
  "Aircraft" varchar(50) NOT NULL,
  "Description" varchar(50) NOT NULL,
  "Max_Gross_Weight" varchar(50) NOT NULL,
  "Total_disk_area" varchar(50) NOT NULL,
  "Max_disk_Loading" varchar(50) NOT NULL,
  PRIMARY KEY (`Aircraft_ID`)
);

CREATE TABLE `match` (
"Round" real,
"Location" text,
"Country" text,
"Date" text,
"Fastest_Qualifying" text,
"Winning_Pilot" text,
"Winning_Aircraft" text,
PRIMARY KEY ("Round"),
FOREIGN KEY (`Winning_Aircraft`) REFERENCES `aircraft`(`Aircraft_ID`),
FOREIGN KEY (`Winning_Pilot`) REFERENCES `pilot`(`Pilot_Id`)
);

CREATE TABLE `airport` (
"Airport_ID" int,
"Airport_Name" text,
"Total_Passengers" real,
"%_Change_2007" text,
"International_Passengers" real,
"Domestic_Passengers" real,
"Transit_Passengers" real,
"Aircraft_Movements" real,
"Freight_Metric_Tonnes" real,
PRIMARY KEY ("Airport_ID")
);

CREATE TABLE `airport_aircraft` (
"ID" int,
"Airport_ID" int,
"Aircraft_ID" int,
PRIMARY KEY ("Airport_ID","Aircraft_ID"),
FOREIGN KEY ("Airport_ID") REFERENCES `airport`(`Airport_ID`),
FOREIGN KEY ("Aircraft_ID") REFERENCES `aircraft`(`Aircraft_ID`)
);

### Answer
Given the database schema, here is the SQLite query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""


architecture_prompt = """
### Task
Generate a SQLite query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Pay close attention to the given database schema, note on how they can join together
- You do not need to generate your thought process but just the answer
- Your answer should end with '[/SQL]'

CREATE TABLE "architect" (
"id" text,
"name" text,
"nationality" text,
"gender" text,
primary key("id")
);

CREATE TABLE "bridge" (
"architect_id" int,
"id" int,
"name" text,
"location" text,
"length_meters" real,
"length_feet" real,
primary key("id"),
foreign key ("architect_id" ) references `architect`("id")
);

CREATE TABLE "mill" (
"architect_id" int,
"id" int,
"location" text,
"name" text,
"type" text,
"built_year" int,
"notes" text,
primary key ("id"),
foreign key ("architect_id" ) references `architect`("id")
);

### Answer
Given the database schema, here is the SQLite query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""


cinema_prompt = """
### Task
Generate a SQLite query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Pay close attention to the given database schema, note on how they can join together
- You do not need to generate your thought process but just the answer
- Your answer should end with '[/SQL]'

CREATE TABLE "film" (
"Film_ID" int,
"Rank_in_series" int,
"Number_in_season" int,
"Title" text,
"Directed_by" text,
"Original_air_date" text,
"Production_code" text,
PRIMARY KEY ("Film_ID")
);

CREATE TABLE "cinema" (
"Cinema_ID" int,
"Name" text,
"Openning_year" int,
"Capacity" int,
"Location" text,
PRIMARY KEY ("Cinema_ID"));

CREATE TABLE "schedule" (
"Cinema_ID" int,
"Film_ID" int,
"Date" text,
"Show_times_per_day" int,
"Price" float,
PRIMARY KEY ("Cinema_ID","Film_ID"),
FOREIGN KEY (`Film_ID`) REFERENCES `film`(`Film_ID`),
FOREIGN KEY (`Cinema_ID`) REFERENCES `cinema`(`Cinema_ID`)
);

### Answer
Given the database schema, here is the SQLite query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

# Store each prompt and its corresponding DB name
dbs = [
    ("department_management", department_prompt),
    ("farm", farm_prompt),
    ("aircraft", aircraft_prompt),
    ("architecture", architecture_prompt),
    ("cinema", cinema_prompt),
]

In [4]:
import sqlite3
import json
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import sqlparse

# Path config
def get_db_path(db_id):
    base_dir = Path(r"C:\Users\zly20\OneDrive - The University of Western Ontario\1B\CS 9860 Advanced Machine Learning\Final Project\CS_9860_Final_Project\data")
    return str(base_dir / f"{db_id}.sqlite")

# Run SQL and return DataFrame
def run_query_on_db(db_path, query):
    try:
        with sqlite3.connect(db_path) as conn:
            result = pd.read_sql_query(query, conn)
        return result
    except Exception as e:
        print("Query failed:", e)
        return None

# Accumulators
correct = 0
total = 0
bad_cases = []

# Per-dataset tracking
correct_by_db = {}
total_by_db = {}

# Loop through each database and use corresponding prompt
for db_id, prompt_template in dbs:
    subset = [item for item in data if item["db_id"] == db_id]
    db_path = get_db_path(db_id)

    correct_local = 0
    total_local = 0

    def generate_query(question):
        prompt = prompt_template.format(question=question)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        generated_ids = model.generate(
            **inputs,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=168,
            do_sample=False,
            num_beams=1,
        )
        outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        return sqlparse.format(outputs[0].split("[SQL]")[1].split("[/SQL]")[0], reindent=True)

    print(f"🔍 Evaluating {db_id} ({len(subset)} questions)")
    for item in tqdm(subset, desc=f"{db_id}"):
        question = item["question"]
        gold_query = item["query"]

        try:
            pred_query = generate_query(question)

            gold_result = run_query_on_db(db_path, gold_query)
            pred_result = run_query_on_db(db_path, pred_query)

            if gold_result is not None and pred_result is not None:
                if gold_result.equals(pred_result):
                    correct += 1
                    correct_local += 1
                else:
                    bad_cases.append({
                        "db_id": db_id,
                        "question": question,
                        "gold_query": gold_query,
                        "pred_query": pred_query,
                        "error_type": "Mismatch",
                        "gold_result": gold_result.to_string(index=False),
                        "pred_result": pred_result.to_string(index=False)
                    })
            else:
                bad_cases.append({
                    "db_id": db_id,
                    "question": question,
                    "gold_query": gold_query,
                    "pred_query": pred_query,
                    "error_type": "ExecutionError",
                    "gold_result": str(gold_result),
                    "pred_result": str(pred_result)
                })

            total += 1
            total_local += 1

        except Exception as e:
            bad_cases.append({
                "db_id": db_id,
                "question": question,
                "gold_query": gold_query,
                "pred_query": "N/A",
                "error_type": f"Exception: {str(e)}",
                "gold_result": "N/A",
                "pred_result": "N/A"
            })
            total += 1
            total_local += 1
            continue

    correct_by_db[db_id] = correct_local
    total_by_db[db_id] = total_local
    print(f"✅ Accuracy for {db_id}: {correct_local}/{total_local} = {correct_local / total_local:.2%}")

safe_model_name = model_name.replace("/", "_")
output_filename = f"bad_cases_{safe_model_name}.json"

with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(bad_cases, f, ensure_ascii=False, indent=2)

print("\n📊 Summary of Accuracy per Dataset:")
for db_id in correct_by_db:
    print(f" - {db_id}: {correct_by_db[db_id]}/{total_by_db[db_id]} = {correct_by_db[db_id] / total_by_db[db_id]:.2%}")

print(f"\n🎯 Final Accuracy: {correct}/{total} = {correct / total:.2%}")
print(f"Saved bad cases to {output_filename}")


🔍 Evaluating department_management (16 questions)


department_management:  44%|████▍     | 7/16 [00:08<00:11,  1.33s/it]

Query failed: Execution failed on sql '
SELECT EXTRACT(YEAR
               FROM d.creation::DATE) AS creation_year
FROM department d
JOIN management m ON d.department_id = m.department_id
JOIN head h ON m.head_id = h.head_id
WHERE h.born_state = 'Alabama'
ORDER BY creation_year NULLS LAST;': near "FROM": syntax error


department_management:  56%|█████▋    | 9/16 [00:11<00:09,  1.41s/it]

Query failed: Execution failed on sql '
SELECT EXTRACT(YEAR
               FROM to_date(d.creation, 'YYYY')) AS year_of_creation,
       COUNT(*) AS number_of_departments
FROM department d
GROUP BY year_of_creation
ORDER BY number_of_departments DESC
LIMIT 1;': near "FROM": syntax error


...

✅ Accuracy for architecture: 5/17 = 29.41%
🔍 Evaluating cinema (30 questions)


cinema:  97%|█████████▋| 29/30 [00:29<00:01,  1.16s/it]

Query failed: Execution failed on sql '
SELECT COUNT(f.Film_ID)
FROM film f
WHERE f.Title ilike '%Dummy%';': near "ilike": syntax error


cinema: 100%|██████████| 30/30 [00:30<00:00,  1.01s/it]

Query failed: Execution failed on sql '
SELECT COUNT(f.Film_ID)
FROM "film" f
WHERE f.Title ilike '%Dummy%';': near "ilike": syntax error
✅ Accuracy for cinema: 14/30 = 46.67%

📊 Summary of Accuracy per Dataset:
 - department_management: 3/16 = 18.75%
 - farm: 17/40 = 42.50%
 - aircraft: 24/46 = 52.17%
 - architecture: 5/17 = 29.41%
 - cinema: 14/30 = 46.67%

🎯 Final Accuracy: 63/149 = 42.28%
Saved bad cases to bad_cases_defog_sqlcoder-7b-2.json
